# 🎯 Caro AI Trainer (Multi-Board Size)
This notebook trains a neural network in PyTorch to play Caro (Tic-Tac-Toe variant) on variable board sizes (3x3 to 10x10), then exports the model to ONNX format for in-browser use.

In [1]:
pip install torch numpy matplotlib onnx

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

BOARD_MAX = 10
INPUT_SIZE = BOARD_MAX * BOARD_MAX
PADDING_VAL = -2  # for non-board cells

class CaroNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(INPUT_SIZE, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, INPUT_SIZE)  # predict score for each cell
        )

    def forward(self, x):
        return self.net(x)

In [3]:
def generate_sample(size):
    board = [PADDING_VAL] * INPUT_SIZE
    flat_board = [0] * (size * size)

    # Simulate a few moves
    num_moves = random.randint(4, (size * size) // 2)
    player = 1  # X starts

    filled = random.sample(range(size * size), num_moves)
    for i, idx in enumerate(filled):
        flat_board[idx] = player
        player *= -1  # alternate turns

    for i in range(size * size):
        board[i] = flat_board[i]

    empty_indices = [i for i in range(size * size) if flat_board[i] == 0]
    if not empty_indices:
        return board, [0.0] * INPUT_SIZE

    move = random.choice(empty_indices)
    label = [0.0] * INPUT_SIZE
    label[move] = 1.0
    return board, label

In [4]:
def train_model():
    model = CaroNet()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.BCEWithLogitsLoss()

    X, y = [], []
    for _ in range(10000):
        size = random.randint(3, 10)
        board, label = generate_sample(size)
        X.append(board)
        y.append(label)

    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)

    for epoch in range(10):
        model.train()
        out = model(X)
        loss = loss_fn(out, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

    torch.save(model.state_dict(), "caro_model.pt")
    print("✅ Saved PyTorch model as caro_model.pt")
    return model

model = train_model()

Epoch 1, Loss: 0.7014
Epoch 2, Loss: 0.6583
Epoch 3, Loss: 0.6165
Epoch 4, Loss: 0.5691
Epoch 5, Loss: 0.5127
Epoch 6, Loss: 0.4472
Epoch 7, Loss: 0.3758
Epoch 8, Loss: 0.3044
Epoch 9, Loss: 0.2402
Epoch 10, Loss: 0.1891
✅ Saved PyTorch model as caro_model.pt


In [5]:
dummy_input = torch.randn(1, INPUT_SIZE)
torch.onnx.export(model, dummy_input, "caro_model.onnx",
                  input_names=["input"], output_names=["output"])
print("✅ Exported model to caro_model.onnx")

✅ Exported model to caro_model.onnx
